<a href="https://colab.research.google.com/github/trinityvthompson/media-bias-ml-project-/blob/main/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install required libraries (if not already installed)
!pip install pandas numpy matplotlib seaborn scipy

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Data handling
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Statistics
from scipy.stats import pearsonr

In [5]:
# Load AllSides dataset
allsides = pd.read_csv('allsides_data.csv')

# Preview first 5 rows
allsides.head()

,news_source,rating,rating_num,type,agree,disagree,perc_agree,url,editorial_review,blind_survey,third_party_analysis,independent_research,confidence_level,twitter,wiki,facebook,screen_name
0,AARP,center,3.0,Think Tank / Policy Group,1404,2343,0.374700,https://www.allsides.com/news-source/aarp-medi...,0,0,0,0,NaN,https://twitter.com/AARP,https://en.wikipedia.org/wiki/AARP,NaN,AARP
1,ABC News,left-center,2.0,News Media,13793,9409,0.594475,https://www.allsides.com/news-source/abc-news-...,0,1,1,1,High,https://twitter.com/ABC,https://en.wikipedia.org/wiki/ABC_News,https://www.facebook.com/ABCNews/,ABC
2,Abridge News,allsides,NaN,News Media,31,17,0.645833,https://www.allsides.com/news-source/abridge-n...,0,0,0,1,High,https://twitter.com/Abridge_News,NaN,https://www.facebook.com/AbridgeNews,Abridge_News
3,Accuracy in Media,right,5.0,Think Tank / Policy Group,743,444,0.625948,https://www.allsides.com/news-source/accuracy-...,0,0,0,1,Medium,NaN,https://en.wikipedia.org/wiki/Accuracy_in_Media,NaN,NaN
4,ACLU,left-center,2.0,Think Tank / Policy Group,1272,1697,0.428427,https://www.allsides.com/news-source/american-...,0,0,0,0,NaN,https://twitter.com/@ACLU,https://en.wikipedia.org/wiki/American_Civil_L...,https://www.facebook.com/aclu/,ACLU


In [6]:
allsides.shape

(547, 17)

In [7]:
# Print column names
print("\nColumn names:")
for col in allsides.columns:
    print("-", col)


Column names:
- news_source
- rating
- rating_num
- type
- agree
- disagree
- perc_agree
- url
- editorial_review
- blind_survey
- third_party_analysis
- independent_research
- confidence_level
- twitter
- wiki
- facebook
- screen_name


In [8]:
# --------------------------------------------
# DATA TYPES (Pandas inferred types)
# --------------------------------------------

print("\nPandas data types:")
print(allsides.dtypes)


Pandas data types:
news_source              object
rating                   object
rating_num              float64
type                     object
agree                     int64
disagree                  int64
perc_agree              float64
url                      object
editorial_review         object
blind_survey              int64
third_party_analysis      int64
independent_research      int64
confidence_level         object
twitter                  object
wiki                     object
facebook                 object
screen_name              object
dtype: object


In [9]:
# --------------------------------------------
# COLUMN NAMES + DATA TYPES + CLASSIFICATION
# --------------------------------------------

summary = []

for col in allsides.columns:

    dtype = allsides[col].dtype

    if dtype == "object":
        classification = "Categorical (Nominal or possibly Ordinal)"
    elif pd.api.types.is_integer_dtype(dtype):
        classification = "Discrete (Quantitative)"
    elif pd.api.types.is_float_dtype(dtype):
        classification = "Continuous (Quantitative)"
    else:
        classification = "Other"

    summary.append([col, str(dtype), classification])

summary_df = pd.DataFrame(summary, columns=[
    "Column Name",
    "Pandas Data Type",
    "Statistical Classification"
])

summary_df

,Column Name,Pandas Data Type,Statistical Classification
0,news_source,object,Categorical (Nominal or possibly Ordinal)
1,rating,object,Categorical (Nominal or possibly Ordinal)
2,rating_num,float64,Continuous (Quantitative)
3,type,object,Categorical (Nominal or possibly Ordinal)
4,agree,int64,Discrete (Quantitative)
5,disagree,int64,Discrete (Quantitative)
6,perc_agree,float64,Continuous (Quantitative)
7,url,object,Categorical (Nominal or possibly Ordinal)
8,editorial_review,object,Categorical (Nominal or possibly Ordinal)
9,blind_survey,int64,Discrete (Quantitative)


In [10]:
# --------------------------------------------
# IDENTIFY CATEGORICAL COLUMNS
# --------------------------------------------

categorical_cols = allsides.select_dtypes(include=["object"]).columns

print("Categorical columns:")
print(categorical_cols)

Categorical columns:
Index(['news_source', 'rating', 'type', 'url', 'editorial_review',
       'confidence_level', 'twitter', 'wiki', 'facebook', 'screen_name'],
      dtype='object')


In [11]:
# --------------------------------------------
# FOR EACH CATEGORICAL COLUMN:
# 1. Discrete values
# 2. Distribution
# 3. Mode
# --------------------------------------------

for col in categorical_cols:

    print("\n=================================================")
    print("Column:", col)

    # Discrete values (unique categories)
    unique_values = allsides[col].dropna().unique()
    print("\nDiscrete values:")
    print(unique_values)

    # Distribution (frequency count)
    distribution = allsides[col].value_counts()
    print("\nDistribution:")
    print(distribution)

    # Mode (most frequent value)
    mode_value = allsides[col].mode()
    print("\nMode (most common value):")
    print(mode_value.values)


Column: news_source

Discrete values:
['AARP' 'ABC News' 'Abridge News' 'Accuracy in Media' 'ACLU' 'AJ+'
 'Al Cardenas' 'Al Jazeera' 'AllSides' 'AllSides Community'
 'Allysia Finley (Wall Street Journal)' 'AlterNet'
 'American Conservative Union' 'American Enterprise Institute'
 'American Greatness' 'American Spectator' 'American Thinker'
 'Americans for Democratic Action' 'Americans for Tax Reform'
 'Andrew Napolitano' 'Andrew Sullivan' 'Andy Ngo' 'Ann Coulter'
 'Annafi Wahed' 'AP' 'Aquinas College Saint' 'Arkadi Gerney'
 'Arkansas Democrat-Gazette' 'Associated Press'
 'Association for Psychological Science' 'Atlanta Journal-Constitution'
 'Atlas Network' 'AtlasOne' 'Austin American-Statesman' 'Axios'
 'AZ Central' 'Barnstable Patriot' 'BBC News' 'Bearing Drift'
 'Ben Sargent (cartoonist)' 'Ben Shapiro' 'Ben Stein' 'Beth Ballentine'
 'Better Angels' 'Billy Binion' 'Bloomberg' 'Blue Virginia'
 'Bob Gorrell (cartoonist)' 'Boing Boing' 'Boston Herald'
 'Boston Herald Editorial' 'Bowling

In [21]:
# Define columns where quantitative stats make sense
# (Excluding binary flags like 'editorial_review' or 'blind_survey')
quant_cols = ['rating_num', 'agree', 'disagree', 'perc_agree']

# Calculate the statistics
stats = allsides[quant_cols].agg(['min', 'max', 'median', 'mean', 'std']).transpose()

# Calculate Range (Max - Min)
stats['range'] = stats['max'] - stats['min']
stats['min'] = stats['min'].astype(int)
stats['max'] = stats['max'].astype(int)

# Rename for clarity and display
stats = stats[['min', 'max', 'range', 'median', 'mean', 'std']]
print("Quantitative Statistics:")
print(stats)

Quantitative Statistics:
            min    max         range      median         mean          std
rating_num    1      5      4.000000    3.000000     2.988743     1.396782
agree         0  29759  29759.000000  206.000000  1250.416819  3396.885664
disagree      0  32082  32082.000000  194.000000  1049.846435  3266.924015
perc_agree    0      1      0.857143    0.532086     0.536285     0.116823


In [22]:
# Identify the binary categorical columns
binary_cols = ['blind_survey', 'editorial_review', 'third_party_analysis', 'independent_research']

for col in binary_cols:
    print(f"\n--- Column: {col} ---")
    print("Discrete Values:", allsides[col].unique())
    print("Distribution:\n", allsides[col].value_counts())
    print("Mode:", allsides[col].mode()[0])


--- Column: blind_survey ---
Discrete Values: [0 1]
Distribution:
 blind_survey
0    530
1     17
Name: count, dtype: int64
Mode: 0

--- Column: editorial_review ---
Discrete Values: ['0' 'checked' '1']
Distribution:
 editorial_review
0          416
checked    106
1           25
Name: count, dtype: int64
Mode: 0

--- Column: third_party_analysis ---
Discrete Values: [0 1]
Distribution:
 third_party_analysis
0    528
1     19
Name: count, dtype: int64
Mode: 0

--- Column: independent_research ---
Discrete Values: [0 1]
Distribution:
 independent_research
1    284
0    263
Name: count, dtype: int64
Mode: 1


In [13]:
import pandas as pd

mbfc_url = "https://raw.githubusercontent.com/trinityvthompson/media-bias-project/refs/heads/main/all_bias_output.csv"
mbfc = pd.read_csv(mbfc_url)

print("MBFC shape:", mbfc.shape)
mbfc.head()

MBFC shape: (599, 5)


,News Title,Domain,Bias,Factual Rating,Credibility Rating
0,1stHeadlines (1stheadlines.com),1stheadlines.com,CENTER,NaN,MEDIUM CREDIBILITY
1,5Pillars (5pillarsuk.com),5pillarsuk.com,CENTER,NaN,HIGH CREDIBILITY
2,9News – KUSA (9news.com),9news.com,CENTER,NaN,HIGH CREDIBILITY
3,11 News – KKCO (nbc11news.com),nbc11news.com,CENTER,NaN,HIGH CREDIBILITY
4,12 News KPNX (12news.com),12news.com,CENTER,HIGH,HIGH CREDIBILITY


In [26]:
# 1. Dataset Size
print(f"Dataset Size: {mbfc.shape[0]} rows, {mbfc.shape[1]} columns")

# 2. Column Names and Types
print("\nColumn Information:")
print(mbfc.info())

# 3. Categorical Analysis
# For MBFC, almost all columns (Bias, Factual Rating, Credibility) are categorical/ordinal.
categorical_cols = mbfc.select_dtypes(include=['object']).columns

for col in categorical_cols:
    print(f"\n" + "="*50)
    print(f"Column: {col}")
    print(f"Discrete Values: {mbfc[col].nunique()} unique values")
    print(f"\nDistribution:\n{mbfc[col].value_counts().head(10)}") # Top 10 shown for brevity
    print(f"\nMode: {mbfc[col].mode()[0]}")

# 4. Quantitative Analysis
# This checks if there are any numerical columns (like 'rating_num' was in AllSides)
quant_cols = mbfc.select_dtypes(include=['int64', 'float64']).columns

if len(quant_cols) > 0:
    print("\n" + "="*50)
    print("Quantitative Statistics:")
    stats = mbfc[quant_cols].agg(['min', 'max', 'median', 'mean', 'std']).transpose()
    stats['range'] = stats['max'] - stats['min']
    print(stats[['range', 'median', 'mean', 'std']])
else:
    print("\nNo quantitative columns found in the MBFC dataset.")

Dataset Size: 599 rows, 5 columns

Column Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 599 entries, 0 to 598
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   News Title          599 non-null    object
 1   Domain              573 non-null    object
 2   Bias                599 non-null    object
 3   Factual Rating      221 non-null    object
 4   Credibility Rating  593 non-null    object
dtypes: object(5)
memory usage: 23.5+ KB
None

Column: News Title
Discrete Values: 599 unique values

Distribution:
News Title
Cumulus Media (cumulusmedia.com/)        1
1stHeadlines (1stheadlines.com)          1
5Pillars (5pillarsuk.com)                1
9News – KUSA (9news.com)                 1
11 News – KKCO (nbc11news.com)           1
12 News KPNX (12news.com)                1
13 News – WIBW (wibw.com)                1
Civitas Institute (www.nccivitas.org)    1
City Journal (www.city-journal.org

In [14]:
# Install libraries we need:
# - warcio: reads WARC web archive files
# - trafilatura: extracts main article text from HTML
# - pandas: dataframes + saving CSV
# - tqdm: progress bar (optional but nice)
!pip -q install warcio trafilatura pandas tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 832.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 7.8 MB/s eta 0:00:00


In [15]:
!pip -q install awscli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 2.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 8.2.3 requires docutils<0.22,>=0.20, but you have docutils 0.19 which is incompatible.


In [16]:
# =========================
# 🔁 CHANGE ONLY THESE TWO
# =========================
YEAR = "2025"
MONTH = "12"   # must be "01".."12"
# =========================

import gzip
import requests
from io import BytesIO

paths_url = f"https://data.commoncrawl.org/crawl-data/CC-NEWS/{YEAR}/{MONTH}/warc.paths.gz"

print("Checking:", paths_url)
r = requests.get(paths_url, timeout=60)
print("Status:", r.status_code)

if r.status_code != 200:
    raise Exception("That YEAR/MONTH doesn't exist for CC-NEWS. Try another month like 11 or 10.")

# unzip and read lines
paths = gzip.decompress(r.content).decode("utf-8").splitlines()
print("Number of WARC files listed:", len(paths))
print("First 5 paths:")
for p in paths[:5]:
    print(p)

Checking: https://data.commoncrawl.org/crawl-data/CC-NEWS/2025/12/warc.paths.gz
Status: 200
Number of WARC files listed: 616
First 5 paths:
crawl-data/CC-NEWS/2025/12/CC-NEWS-20251201001011-05528.warc.gz
crawl-data/CC-NEWS/2025/12/CC-NEWS-20251201014657-05529.warc.gz
crawl-data/CC-NEWS/2025/12/CC-NEWS-20251201031935-05530.warc.gz
crawl-data/CC-NEWS/2025/12/CC-NEWS-20251201044518-05531.warc.gz
crawl-data/CC-NEWS/2025/12/CC-NEWS-20251201060253-05532.warc.gz


In [17]:
import os

os.makedirs("ccnews_warc_sample", exist_ok=True)

sample_paths = paths[:3]  # change to [:10], [:50], etc.
for p in sample_paths:
    url = "https://data.commoncrawl.org/" + p
    out = os.path.join("ccnews_warc_sample", os.path.basename(p))
    print("Downloading:", url)
    with requests.get(url, stream=True, timeout=300) as resp:
        resp.raise_for_status()
        with open(out, "wb") as f:
            for chunk in resp.iter_content(chunk_size=1024*1024):
                if chunk:
                    f.write(chunk)
    print("Saved:", out)

Downloading: https://data.commoncrawl.org/crawl-data/CC-NEWS/2025/12/CC-NEWS-20251201001011-05528.warc.gz
Saved: ccnews_warc_sample/CC-NEWS-20251201001011-05528.warc.gz
Downloading: https://data.commoncrawl.org/crawl-data/CC-NEWS/2025/12/CC-NEWS-20251201014657-05529.warc.gz
Saved: ccnews_warc_sample/CC-NEWS-20251201014657-05529.warc.gz
Downloading: https://data.commoncrawl.org/crawl-data/CC-NEWS/2025/12/CC-NEWS-20251201031935-05530.warc.gz
Saved: ccnews_warc_sample/CC-NEWS-20251201031935-05530.warc.gz


In [18]:
!pip -q install warcio trafilatura

In [27]:
import gzip
import logging
from warcio.archiveiterator import ArchiveIterator
import trafilatura
import pandas as pd
import os

# ✅ reduce warning noise
logging.getLogger("trafilatura").setLevel(logging.ERROR)

folder = "ccnews_warc_sample"
files = [f for f in os.listdir(folder) if f.endswith(".warc.gz")]

rows = []

for filename in files:
    path = os.path.join(folder, filename)
    print("Processing:", filename)

    with gzip.open(path, "rb") as stream:
        for record in ArchiveIterator(stream):
            if record.rec_type != "response":
                continue

            url = record.rec_headers.get_header("WARC-Target-URI")
            date = record.rec_headers.get_header("WARC-Date")

            try:
                html = record.content_stream().read()

                # ✅ trafilatura likes bytes too
                downloaded = trafilatura.extract(
                    html,
                    output_format="txt",
                    include_comments=False,
                    include_tables=False,
                    favor_precision=False,
                    favor_recall=True
                )

                if downloaded and len(downloaded.split()) >= 200:
                    rows.append({"url": url, "date": date, "text": downloaded})

            except Exception:
                continue

df = pd.DataFrame(rows)
print("Dataset shape:", df.shape)
df.head()

Processing: CC-NEWS-20251201014657-05529.warc.gz


ERROR:trafilatura.utils:parsed tree length: 1, wrong data type or not valid HTML
ERROR:trafilatura.core:empty HTML tree: None


Processing: CC-NEWS-20251201001011-05528.warc.gz


KeyboardInterrupt: 

In [24]:
commoncrawl = df

# Assuming your dataframe is named 'commoncrawl'
commoncrawl.to_csv('/content/drive/MyDrive/commoncrawl_data.csv', index=False)

print("File saved successfully to Google Drive!")

NameError: name 'df' is not defined

In [28]:
import pandas as pd

df = pd.read_csv("left_vs_right_articles_2025.csv")

print("Dataset shape:", df.shape)
df.head()

EmptyDataError: No columns to parse from file

### Data Wrangling

In [ ]:
# Check for missing values in each column
missing_data = allsides.isnull().sum()
print("Missing data per column:")
print(missing_data[missing_data > 0])

In [ ]:
# Remove any rows w/o a news_source name or a rating category - not useful for bias analysis
initial_shape = allsides.shape
allsides.dropna(subset=['news_source', 'rating'], inplace=True)

print(f"Rows removed: {initial_shape[0] - allsides.shape[0]}")

In [ ]:
# Replace missing rating_num with the median
allsides['rating_num'] = allsides['rating_num'].fillna(allsides['rating_num'].median())

# Replace missing confidence_level with an 'Unknown' placeholder
allsides['confidence_level'] = allsides['confidence_level'].fillna('Unknown')

# Replace missing social media links with 'None'
for col in ['twitter', 'wiki', 'facebook']:
    allsides[col] = allsides[col].fillna('None')

In [ ]:
# Ensure rating_num is within 1-5
out_of_range_rating = allsides[(allsides['rating_num'] < 1) | (allsides['rating_num'] > 5)]
print(f"Invalid ratings found: {len(out_of_range_rating)}")

# Ensure perc_agree is within 0-1
out_of_range_perc = allsides[(allsides['perc_agree'] < 0) | (allsides['perc_agree'] > 1)]
print(f"Invalid percentages found: {len(out_of_range_perc)}")

In [ ]:
# Convert rating_num to float if it isn't already
allsides['rating_num'] = pd.to_numeric(allsides['rating_num'], errors='coerce')

# Clean string columns to prevent formatting issues
string_cols = allsides.select_dtypes(include=['object']).columns
for col in string_cols:
    allsides[col] = allsides[col].str.strip()

print("Data formats verified.")

In [ ]:
# Check for duplicate news sources
duplicate_count = allsides.duplicated(subset=['news_source', 'url']).sum()
print(f"Duplicate rows found: {duplicate_count}")

# Remove duplicates if they exist
if duplicate_count > 0:
    allsides.drop_duplicates(subset=['news_source', 'url'], keep='first', inplace=True)
    print("Duplicates removed.")

### Preliminary Statistical Analysis

In [ ]:
# Select only quantitative columns
quantitative_data = allsides.select_dtypes(include=[np.number])

# Calculate Pearson correlation matrix
correlation_matrix = quantitative_data.corr(method='pearson')

# Display matrix
print("Pearson Correlation Coefficients:")
display(correlation_matrix)

# Visualization: Heatmap to easily spot high correlations
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Heatmap of Quantitative Data")
plt.show()

In [ ]:
from sklearn.metrics import r2_score

# Define the variables to test
x = allsides['agree']
y = allsides['disagree']

# Define different degrees to test: Linear, Quadratic, Cubic
degrees = [1, 2, 3]
names = ['Linear', 'Quadratic', 'Cubic']

print("Model Fit Comparison (R² Scores):")
for degree, name in zip(degrees, names):
    # Calculate the coefficients
    coeffs = np.polyfit(x, y, degree)
    # Create the model function
    model = np.poly1d(coeffs)
    # Calculate R-squared
    score = r2_score(y, model(x))
    print(f"{name} Fit: {score:.4f}")

In [ ]:
# Visualize the Best Fit (cubic)
plt.figure(figsize=(8, 5))
plt.scatter(x, y, alpha=0.5, label='Actual Data')
x_range = np.linspace(x.min(), x.max(), 100)
plt.plot(x_range, np.poly1d(np.polyfit(x, y, 3))(x_range), color='red', label='Cubic Fit Line')
plt.xlabel('Agree Votes')
plt.ylabel('Disagree Votes')
plt.legend()
plt.title("Best Fit Determination: Agree vs. Disagree")
plt.show()